# Project 2 - Social Network Mining

## Part 1 - Facebook network

In [2]:
# # Install Necessary Packages
# install.packages("igraph")
# install.packages("Matrix")
# install.packages("pracma")
# install.packages("entropy")
# install.packages('infotheo')

# Load Packages 
library('igraph')
library('Matrix')
library('pracma')
library('igraph')
library('entropy')
library('infotheo')


Attaching package: ‘igraph’


The following objects are masked from ‘package:stats’:

    decompose, spectrum


The following object is masked from ‘package:base’:

    union



Attaching package: ‘pracma’


The following objects are masked from ‘package:Matrix’:

    expm, lu, tril, triu



Attaching package: ‘infotheo’


The following objects are masked from ‘package:entropy’:

    discretize, entropy




In [20]:
# Load the facebook network graph
data_frame_fb_network = as.matrix(read.table('data/facebook_combined.txt', header = FALSE, sep = " ")) + 1
graph_fb_network = graph_from_edgelist(data_frame_fb, directed = FALSE)

**QUESTION 16:** What is |Nr|, i.e. the length of the list Nr?

In [45]:
# get neighbors of node ID 415
neighbor_ids = as_ids(neighbors(graph_fb_network, 415, mode = "all"))
neighbor_ids = append(neighbor_ids, 415)

# create a sub-graph
node_415_sub_graph = induced_subgraph(graph_fb_network, neighbor_ids)

# get users with degree greater than 24. 
degree_list_all = degree(node_415_sub_graph, mode = "all")
degree_list_24 = which(degree_list_all == 24)
N_r = length(degree_list_24)

# print Nr
print("Number of users with degree 24 : ")
N_r
print("Users of node 415 with degree 24 : ")
degree_list_24


[1] "Number of users with degree 24 : "


[1] 11

[1] "Users of node 415 with degree 24 : "


[1]  31  53  75  90  93 102 118 133 134 136 137

**QUESTION 17:** Compute the average accuracy of the friend recommendation algorithm that
uses:

• Common Neighbors measure

• Jaccard measure

• Adamic Adar measure

Based on the average accuracy values, which friend recommendation algorithm is the best?

Hint Useful function(s): similarity

In [59]:
# Average Accuracy of Common Neighbors measure
avg_accuracy_users = c()
max_steps = 10
for(node_i in degree_list_24){
    
    iteration_accuracy = c()
    for (step in c(1:max_steps)){
        
        iteration_graph = node_415_sub_graph
        R_i = c() # list of friends deleted
        neighbors_of_node_i = as_ids(neighbors(iteration_graph, node_i, mode = "all"))
        for (node in neighbors_of_node_i){
            
            if (runif(1, 0, 1) <= 0.25){
                
                iteration_graph = delete_edges(iteration_graph, edge(node, node_i)) # delete nodes at ranodom. 
                R_i = append(R_i, node)
            }
        }
        node_i_current_nighbors = setdiff(neighbors_of_node_i, R_i)
        node_i_current_nighbors = append(node_i_current_nighbors, node_i)
        node_i_neighbors_compliment = setdiff(V(iteration_graph), node_i_current_nighbors)
        common_neighbors_measure = c()
        for (node_j in node_i_neighbors_compliment){
            
            neighbors_of_node_j = as_ids(neighbors(iteration_graph, node_j, mode = "all"))
            common_neighbors_nodei_nodej = intersect(node_i_current_nighbors, neighbors_of_node_j)
            common_neighbors_measure = c(common_neighbors_measure, length(common_neighbors_nodei_nodej))
        }
        index = sort(common_neighbors_measure,decreasing=TRUE, index.return=TRUE)$ix
        P_i = node_i_neighbors_compliment[index[1:length(R_i)]] # get the top |R_i| recommendations. 
        Pi_intersect_Ri = intersect(R_i, P_i) # P_i intersect R_i
        iteration_accuracy = append(iteration_accuracy, length(Pi_intersect_Ri)/length(R_i)) # accuracy of prediction for user i in one iterartion

    }
    avg_accuracy_users = append(avg_accuracy_users,mean(iteration_accuracy)) # average accuracy of prediction for user i
    print(sprintf("Average accuarcy of Commmon neighbors measure recommendation for user %d is %3.8f", node_i,mean(iteration_accuracy)))  
}
avg_accuracy_common_neighbours_measure = mean(avg_accuracy_users) # average accuracy of prediction for the recommendation algorithm. 
cat("\n")
print(sprintf("Average accuarcy of Commmon neighbors measure is %3.8f",avg_accuracy_common_neighbours_measure))



[1] "Average accuarcy of Commmon neighbors measure recommendation for user 31 is 0.47486291"
[1] "Average accuarcy of Commmon neighbors measure recommendation for user 53 is 1.00000000"
[1] "Average accuarcy of Commmon neighbors measure recommendation for user 75 is 0.92250000"
[1] "Average accuarcy of Commmon neighbors measure recommendation for user 90 is 0.79916667"
[1] "Average accuarcy of Commmon neighbors measure recommendation for user 93 is 0.43107143"
[1] "Average accuarcy of Commmon neighbors measure recommendation for user 102 is 1.00000000"
[1] "Average accuarcy of Commmon neighbors measure recommendation for user 118 is 0.92543651"
[1] "Average accuarcy of Commmon neighbors measure recommendation for user 133 is 1.00000000"
[1] "Average accuarcy of Commmon neighbors measure recommendation for user 134 is 0.98888889"
[1] "Average accuarcy of Commmon neighbors measure recommendation for user 136 is 0.90119048"
[1] "Average accuarcy of Commmon neighbors measure recommendation

In [62]:
# Average Accuracy of Jaccard Measure measure
avg_accuracy_users = c()
max_steps = 10
for(node_i in degree_list_24){
    
    iteration_accuracy = c()
    for (step in c(1:max_steps)){
        
        iteration_graph = node_415_sub_graph
        R_i = c() # list of friends deleted
        neighbors_of_node_i = as_ids(neighbors(iteration_graph, node_i, mode = "all"))
        for (node in neighbors_of_node_i){
            if (runif(1, 0, 1) <= 0.25){
                iteration_graph = delete_edges(iteration_graph, edge(node, node_i)) # delete nodes at ranodom.
                R_i = append(R_i, node)
            }
        }
        node_i_current_nighbors = setdiff(neighbors_of_node_i, R_i)
        node_i_current_nighbors = append(node_i_current_nighbors, node_i)
        node_i_neighbors_compliment = setdiff(V(iteration_graph), node_i_current_nighbors)
        jaccard_measure = c()
        for (node_j in node_i_neighbors_compliment){
            
            neighbors_of_node_j = as_ids(neighbors(iteration_graph, node_j, mode = "all"))
            common_neighbors_nodei_nodej = intersect(node_i_current_nighbors, neighbors_of_node_j)
            all_neighbors_nodei_nodej = union_common_neighbours = union(node_i_current_nighbors, neighbors_of_node_j)
            jaccard_measure = c(jaccard_measure, length(common_neighbors_nodei_nodej)/length(all_neighbors_nodei_nodej))
        }
        index = sort(jaccard_measure,decreasing=TRUE, index.return=TRUE)$ix
        P_i = node_i_neighbors_compliment[index[1:length(R_i)]] # get the top |R_i| recommendations. 
        Pi_intersect_Ri = intersect(R_i, P_i) # P_i intersect R_i
        iteration_accuracy = append(iteration_accuracy, length(Pi_intersect_Ri)/length(R_i)) # accuracy of prediction for user i in one iterartion
        #     print(length(R_i))
    }
    avg_accuracy_users = append(avg_accuracy_users,mean(iteration_accuracy)) # average accuracy of prediction for user i
    print(sprintf("Average accuarcy of Jaccard measure recommendation for user %d is %3.8f", node_i,mean(iteration_accuracy)))  
}
avg_accuracy_jaccard_measure = mean(avg_accuracy_users) # average accuracy of prediction for the recommendation algorithm. 
cat("\n")
print(sprintf("Average accuarcy of Jaccard measure is %3.8f",avg_accuracy_jaccard_measure))



[1] "Average accuarcy of Jaccard measure recommendation for user 31 is 0.26323593"
[1] "Average accuarcy of Jaccard measure recommendation for user 53 is 0.98333333"
[1] "Average accuarcy of Jaccard measure recommendation for user 75 is 0.85884560"
[1] "Average accuarcy of Jaccard measure recommendation for user 90 is 0.80321429"
[1] "Average accuarcy of Jaccard measure recommendation for user 93 is 0.65884921"
[1] "Average accuarcy of Jaccard measure recommendation for user 102 is 0.94722222"
[1] "Average accuarcy of Jaccard measure recommendation for user 118 is 0.88960317"
[1] "Average accuarcy of Jaccard measure recommendation for user 133 is 0.96571429"
[1] "Average accuarcy of Jaccard measure recommendation for user 134 is 0.91666667"
[1] "Average accuarcy of Jaccard measure recommendation for user 136 is 0.89626984"
[1] "Average accuarcy of Jaccard measure recommendation for user 137 is 0.96888889"

[1] "Average accuarcy of Jaccard measure is 0.83198577"


In [63]:
# Average Accuracy of Adamic-Adar Measure measure
avg_accuracy_users = c()
max_steps = 10
for(node_i in degree_list_24){
    
    iteration_accuracy = c()
    for (step in c(1:max_steps)){
        
        iteration_graph = node_415_sub_graph
        R_i = c() # list of friends deleted
        neighbors_of_node_i = as_ids(neighbors(iteration_graph, node_i, mode = "all"))
        for (node in neighbors_of_node_i){
            
            if (runif(1, 0, 1) <= 0.25){
                
                iteration_graph = delete_edges(iteration_graph, edge(node, node_i)) # delete nodes at ranodom.
                R_i = append(R_i, node)
            }
        }
        node_i_current_nighbors = setdiff(neighbors_of_node_i, R_i)
        node_i_current_nighbors = append(node_i_current_nighbors, node_i)
        node_i_neighbors_compliment = setdiff(V(iteration_graph), node_i_current_nighbors)
        adamic_adar_measure = c()
        for (node_j in node_i_neighbors_compliment){
            
            neighbors_of_node_j = as_ids(neighbors(iteration_graph, node_j, mode = "all"))
            common_neighbors_nodei_nodej = intersect(node_i_current_nighbors, neighbors_of_node_j)
            log_term_k = 0
            for (neighbour_k in common_neighbors_nodei_nodej){
                
                neighbour_size_k = length(neighbors(iteration_graph, neighbour_k, mode = "all"))
                log_term_k = log_term_k + 1/log10(neighbour_size_k)
            }
            adamic_adar_measure = c(adamic_adar_measure, log_term_k)
        }
        index = sort(adamic_adar_measure,decreasing=TRUE, index.return=TRUE)$ix
        P_i = node_i_neighbors_compliment[index[1:length(R_i)]] # get the top |R_i| recommendations. 
        Pi_intersect_Ri = intersect(R_i, P_i) # P_i intersect R_i
        iteration_accuracy = append(iteration_accuracy, length(Pi_intersect_Ri)/length(R_i)) # accuracy of prediction for user i in one iterartion
        #     print(length(R_i))
    }
    avg_accuracy_users = append(avg_accuracy_users,mean(iteration_accuracy)) # average accuracy of prediction for user i
    print(sprintf("Average accuarcy of Adamic-Adar measure recommendation for user %d is %3.8f", node_i,mean(iteration_accuracy)))  
}
avg_accuracy_adamic_adar_measure = mean(avg_accuracy_users) # average accuracy of prediction for the recommendation algorithm. 
cat("\n")
print(sprintf("Average accuarcy of Adamic-Adar measure is %3.8f",avg_accuracy_adamic_adar_measure))



[1] "Average accuarcy of Adamic-Adar measure recommendation for user 31 is 0.47460317"
[1] "Average accuarcy of Adamic-Adar measure recommendation for user 53 is 0.99000000"
[1] "Average accuarcy of Adamic-Adar measure recommendation for user 75 is 0.93555556"
[1] "Average accuarcy of Adamic-Adar measure recommendation for user 90 is 0.83849206"
[1] "Average accuarcy of Adamic-Adar measure recommendation for user 93 is 0.39285714"
[1] "Average accuarcy of Adamic-Adar measure recommendation for user 102 is 0.99000000"
[1] "Average accuarcy of Adamic-Adar measure recommendation for user 118 is 0.90158730"
[1] "Average accuarcy of Adamic-Adar measure recommendation for user 133 is 1.00000000"
[1] "Average accuarcy of Adamic-Adar measure recommendation for user 134 is 1.00000000"
[1] "Average accuarcy of Adamic-Adar measure recommendation for user 136 is 0.86472222"
[1] "Average accuarcy of Adamic-Adar measure recommendation for user 137 is 0.97083333"

[1] "Average accuarcy of Adamic-Adar